<a href="https://colab.research.google.com/github/AzadSing/AniketTheGreat.github.io/blob/main/Azad_Final_Project_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import random
import string
import json
import binascii
import numpy as np 
import pandas as pd
import pylab as pl
import logging
import datetime
import collections

In [ ]:
!pip install Crypto
!pip install crypto
!pip install pycrypto

# New Section

In [ ]:
!pip install ecpy

In [ ]:
import Crypto
import Crypto.Random

from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5
from ecpy.curves import Curve
from ecpy.keys import ECPrivateKey
from ecpy.ecdsa      import ECDSA
import secrets, binascii


In [ ]:

class Client:
    def __init__(self):
      random = Crypto.Random.new().read
      self._private_key = RSA.generate(1024, random)
      self._public_key = self._private_key.publickey()
      self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
      return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
    


In [ ]:
## Extended Euclidean Algo for Integers
from fractions import Fraction as frac
from operator import add
from operator import neg
from operator import mod


#Necessary Helper functions for NTRU



def modPoly(c,k):
  if(k==0):
    print("Error in modPoly(c,k). Integer k must be non-zero")
  else:
    return [fracMod(x,k) for x in c]

def subPoly(c1,c2):
  [c1,c2]=resize(c1,c2) 
  c2=list(map(neg,c2))
  out=list(map(add, c1, c2))
  return trim(out)

def multPoly(c1,c2):
  order=(len(c1)-1+len(c2)-1)
  out=[0]*(order+1)
  for i in range(0,len(c1)):
    for j in range(0,len(c2)):
      out[j+i]=out[j+i]+c1[i]*c2[j]
  return trim(out)

def resize(c1,c2):
  if(len(c1)>len(c2)):
      c2=c2+[0]*(len(c1)-len(c2))
  if(len(c1)<len(c2)):
      c1=c1+[0]*(len(c2)-len(c1))
  return [c1,c2]

def trim(seq):
  if len(seq) == 0:
      return seq
  else:
      for i in range(len(seq) - 1, -1, -1):
          if seq[i] != 0:
              break
  return seq[0:i+1]

def extEuclidPoly(a,b):
  switch = False
  a=trim(a)
  b=trim(b)
  if len(a)>=len(b):
    a1, b1 = a, b
  else:
    a1, b1 = b, a
    switch = True
  Q,R=[],[]
  while b1 != [0]:
    [q,r]=divPoly(a1,b1)
    Q.append(q)
    R.append(r)
    a1=b1
    b1=r
  S=[0]*(len(Q)+2)
  T=[0]*(len(Q)+2)

  S[0],S[1],T[0],T[1] = [1],[0],[0],[1]

  for x in range(2, len(S)):
    S[x]=subPoly(S[x-2],multPoly(Q[x-2],S[x-1]))
    T[x]=subPoly(T[x-2],multPoly(Q[x-2],T[x-1]))

  gcdVal=R[len(R)-2]
  s_out=S[len(S)-2]
  t_out=T[len(T)-2]
    ### ADDITIONAL STEPS TO SCALE GCD SUCH THAT LEADING TERM AS COEF OF 1:
  scaleFactor=gcdVal[len(gcdVal)-1]
  gcdVal=[x/scaleFactor for x in gcdVal]
  s_out=[x/scaleFactor for x in s_out]
  t_out=[x/scaleFactor for x in t_out]
    
  if switch:
    return [gcdVal,t_out,s_out]
  else:
    return [gcdVal,s_out,t_out]

def divPoly(N,D):
    N, D = list(map(frac,trim(N))), list(map(frac,trim(D)))
    degN, degD = len(N)-1, len(D)-1
    if(degN>=degD):
        q=[0]*(degN-degD+1)
        while(degN>=degD and N!=[0]):
            d=list(D)
            [d.insert(0,frac(0,1)) for i in range(degN-degD)]
            q[degN-degD]=N[degN]/d[len(d)-1]
            d=[x*q[degN-degD] for x in d]
            N=subPoly(N,d)
            degN=len(N)-1
        r=N 
    else:
        q=[0]
        r=N
    return [trim(q),trim(r)]

def addPoly(c1,c2):
    [c1,c2]=resize(c1,c2)
    out=list(map(add, c1, c2))
    return trim(out)

def cenPoly(c,q):
    u=float(q)/float(2)
    l=-u
    c=modPoly(c,q)
    c=[mod(x,-q) if x>u else x for x in c]
    c=[mod(x,q) if x<=l else x for x in c]
    return c

def reModulo(num,div,modby):
    [_,remain]=divPoly(num,div)
    return modPoly(remain,modby)

def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b % a
        m, n = x-u * q, y-v * q
        b,a, x,y, u,v = a,r, u,v, m,n
    gcdVal = b
    return gcdVal, x, y


#Modular inverse
#An application of extended GCD algorithm to finding modular inverses:
def modinv(a, m):
    gcdVal, x, y = egcd(a, m)
    if gcdVal != 1:
        return None  # modular inverse does not exist
    else:
        return x % m

#Modulus Function which handles Fractions aswell
def fracMod(f,m):
    [tmp,t1,t2]=egcd(f.denominator,m)
    if tmp!=1:
        print ("ERROR GCD of denominator and m is not 1")
        1/0
    else:
        out=modinv(f.denominator,m)*f.numerator % m
        return out

In [ ]:
##NTRU operations
N=11
p=3
q=32

f=[-1,1,1,0,-1,0,1,0,0,1,-1]
g=[-1,0,1,1,0,1,0,0,-1,0,-1]


print("==== Bob generates public key =====")
print("Values used:")
print(" N=",N)
print(" p=",p)
print(" q=",q)
print("========")
print("\nBob picks two polynomials (g and f):")


#f=[-1,0,1,1,-1,0,-1]
#g=[0,-1,-1,0,1,0,1]


print("f(x)= ",f)
print("g(x)= ",g)



D=[0]*(N+1)
D[0]=-1
D[N]=1


print("\n====Now we determine F_p and F_q ===")
[gcd_f,s_f,t_f]=extEuclidPoly(f,D)

f_p=modPoly(s_f,p)
f_q=modPoly(s_f,q)
print("F_p:",f_p)
print("F_q:",f_q)

x=multPoly(f_q,g)
h=reModulo(x,D,q)

print("\n====And finally h====")
print("f_q x g: ",x)
print("H (Bob's Public Key): ",h)



==== Bob generates public key =====
Values used:
 N= 11
 p= 3
 q= 32

Bob picks two polynomials (g and f):
f(x)=  [-1, 1, 1, 0, -1, 0, 1, 0, 0, 1, -1]
g(x)=  [-1, 0, 1, 1, 0, 1, 0, 0, -1, 0, -1]

====Now we determine F_p and F_q ===
F_p: [1, 2, 0, 2, 2, 1, 0, 2, 1, 2, 0]
F_q: [5, 9, 6, 16, 4, 15, 16, 22, 20, 18, 30]

====And finally h====
f_q x g:  [-5, -9, -1, -2, 11, 12, 13, 3, 22, 15, 16, 29, 60, 19, -2, -7, -36, -40, -50, -18, -30]
H (Bob's Public Key):  [24, 19, 18, 28, 4, 8, 5, 17, 4, 17, 16]


In [ ]:
##NTRU Encryption and Decryption


print("\n====Let's encrypt====")
#msg=[1,0,1,0,1,1,1]
randPol=[-1,-1,1,1]

#print("Alice's Message:\t",msg)
print("Random:\t\t\t",randPol)
#####################################################################

print(" Enter the message to be Encrypted  ")
data = input()


def NTRUEncryption(data, h, N, p, q, D):
  plaintext = []
  ciphertext = []

  for i in data:
    element = (format(ord(i),'b'))
    element = element[::-1]
    res = [int(x) for x in str(element)]
    plaintext.append(res)
    e_tilda=addPoly(multPoly(multPoly([p],randPol),h),res)
    e = reModulo(e_tilda,D,q)
    ciphertext.append(e)
  print("Original message is: \t ",plaintext)
  return ciphertext

ciphertext = NTRUEncryption(data, h, N, p, q, D)


print("Encrypted message:\t",ciphertext)



######################################################################
print("\n====Let's decrypt====")

recovered = []

def convertToBinary(l):
  l = [str(i) for i in l]
  binaryString = ""
  for i in l:
    binaryString += i
  return int(binaryString, 2)

def removeZeroes(l):
  for i in range(len(l)):
    if l[i] != 0:
      l = l[i:]
      break
  return l


def NTRUDecryption(ciphertext, f, f_p, N, p, q, D):
  decrypted_msg = ''
  for e in ciphertext:
    tmp=reModulo(multPoly(f,e),D,q)
    centered=cenPoly(tmp,q)
    m1=multPoly(f_p,centered)
    tmp=reModulo(m1,D,p)
    recovered.append(tmp)
    tmp= tmp[::-1]
    decrypted_msg += chr(convertToBinary(removeZeroes(tmp)))
  print("Decrypted message:\t",recovered)
  return decrypted_msg
	

    


print("Decrypted String:\t", NTRUDecryption(ciphertext, f, f_p, N, p, q, D))


====Let's encrypt====
Random:			 [-1, -1, 1, 1]
 Enter the message to be Encrypted  
nitw
Original message is: 	  [[0, 1, 1, 1, 0, 1, 1], [1, 0, 0, 1, 0, 1, 1], [0, 0, 1, 0, 1, 1, 1], [1, 1, 1, 0, 1, 1, 1]]
Encrypted message:	 [[7, 3, 10, 24, 15, 7, 26, 2, 8, 3, 28], [8, 2, 9, 24, 15, 7, 26, 2, 8, 3, 28], [7, 2, 10, 23, 16, 7, 26, 2, 8, 3, 28], [8, 3, 10, 23, 16, 7, 26, 2, 8, 3, 28]]

====Let's decrypt====
Decrypted message:	 [[0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0], [1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1, 1, 0, 0, 0]]
Decrypted String:	 nitw


In [ ]:
class ClientNTRU:
  def __init__(self):
    N=11
    p=3
    q=32
    f=[-1,1,1,0,-1,0,1,0,0,1,-1]
    g=[-1,0,1,1,0,1,0,0,-1,0,-1]
    D=[0]*(N+1)
    D[0]=-1
    D[N]=1
    [gcd_f,s_f,t_f]=extEuclidPoly(f,D)
    f_p=modPoly(s_f,p)
    f_q=modPoly(s_f,q)
    self._public_key = (f, f_p, N, p, q, D)
    x=multPoly(f_q,g)
    h=reModulo(x,D,q)
    self._private_key = (h, N, p, q, D)

  @property
  def identity(self):
    return self._public_key
  
      

In [ ]:
class ClientECDSA:
    def __init__(self):
      curve = Curve.get_curve('secp256k1')
      signer = ECDSA()
      self._private_key = ECPrivateKey(secrets.randbits(32*8), curve)
      self._public_key = self._private_key.get_public_key()

    @property
    def identity(self):
      return binascii.hexlify((hex(self._public_key.W.x) + hex(self._public_key.W.y)).encode()).decode('ascii')

In [ ]:
print("Using RSA CryptoSystem")
shiv= Client()
vishnu= Client()
print("shiv's public key is :",shiv.identity)
print("vishnu's public key is :",vishnu.identity)

print("Using ECC Cryptosystem")
Ravi = ClientECDSA()
Azad = ClientECDSA()
print("Ravi's public key is :",Ravi.identity)
print("Azad's public key is :",Azad.identity)

print("Using NTRU Cryptosystem")
ram = ClientNTRU()
shyam = ClientNTRU()
print("Ram's public key is :",ram.identity)
print("Shyam's public key is :",shyam.identity)

Using RSA CryptoSystem
shiv's public key is : 30819f300d06092a864886f70d010101050003818d0030818902818100efb7b5eebfd3071da29b6f64d93239153be56a70d8f4749004f762e052042ae712a43db98a8a3e6a07149afb1d917618f7c091b40113c4a99b6705bfc6070101e533c6e3d574ac1c36b6103caa7b8739c2ae58e67a1c2beddf72a1b73bb0616ca3f87ddba495ef0c9bc3b842ff3f835f139cee81305323a81f439f90e88a3b910203010001
vishnu's public key is : 30819f300d06092a864886f70d010101050003818d0030818902818100ac8dc6df693765d63d16f88e17d9f778e4810357ae242e119246ae56f199dc0a661a98b3ef48dd48cd13897f3b8e0bf1509addce97b20aa550417454b2b206b823b92cbc7ed5bda7caad0802fb669c95d39971357732a31c56e1a0814eec8f2146b2728d788fcd569a45c1894a17e901e9597aab2ca038de0504ae2f9002fc830203010001
Using ECC Cryptosystem
Ravi's public key is : 30786339303438646133386634656166613233303865653135363664383863356163623562383461373635343135313463313766303337646334653565303733386530786339366331313837666238616264646433356263643933653964373162613035396436643466326566313166343332303

In [ ]:
class Transaction:
  def __init__(self, sender, recipient, value):
    self.sender= sender
    self.recipient = recipient
    self.value= value
    self.time = datetime.datetime.now()
  
  def to_dict(self):
    if self.sender == "Genesis":
      identity = "Genesis"
    else:
      identity = self.sender.identity
    
    return collections.OrderedDict({
        'sender' : identity,
        'recipient' : self.recipient,
        'value' : self.value,
        'time' : self.time
    })

  def sign_transaction(self):
    private_key = self.sender._private_key
    signer = PKCS1_v1_5.new(private_key)
    h = SHA.new(str(self.to_dict()).encode('utf8'))
    return binascii.hexlify(signer.sign(h)).decode('ascii')
  
  def sign_transactionECDSA(self):
    h = SHA.new(str(self.to_dict()).encode('utf8'))
    # print(type(h.hexdigest()))
    signer = ECDSA()
    return binascii.hexlify(signer.sign(h.digest(), self.sender._private_key)).decode('ascii')

  def sign_transactionNTRU(self):
    hash = SHA.new(str(self.to_dict()).encode('utf8'))
    private_key = self.sender._private_key
    (h, N, p, q, D) = private_key
    return NTRUEncryption(str(hash.digest()), h, N, p, q, D)








#testing the transaction class

In [ ]:
t1 = Transaction(shiv, vishnu.identity , 15)
%timeit signature_t1 = t1.sign_transaction()
# print(signature_t1)

t2 = Transaction(Azad, Ravi.identity , 15) #the t is new transaction
%timeit signature_t2 = t2.sign_transactionECDSA()
# print(signature_t2)

t3 = Transaction(ram, shyam.identity , 15) #the t is new transaction
%timeit signature_t3 = t3.sign_transactionNTRU()
# print(signature_t3)



100 loops, best of 5: 2.92 ms per loop
100 loops, best of 5: 5.17 ms per loop
Original message is: 	  [[0, 1, 0, 0, 0, 1, 1], [1, 1, 1, 0, 0, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1], [1, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1], [1, 0, 0, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [0, 0, 1, 0, 0, 1, 1], [1, 0, 1, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1], [1, 0, 0, 1, 1, 1], [1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 0, 0, 1, 1], [0, 0, 1, 0, 0, 1, 1], [1, 1, 0, 1, 1, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1], [1, 0, 1, 0, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1], [1, 0, 1, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [0, 0, 1, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [1, 0, 

 **inorder to display the transaction**

In [ ]:
def display_transaction(transaction):
  for transaction in transactions:
    dict = transaction.to_dict()
    print(dict)

transactions = []
transactionsRSA = []
transactionsEDCSA = []
transactionsNTRU = []

    

In [ ]:

#print(Ram.identity ,"\n", Laxman.identity,"\n",Bharath.identity,"\n", Shatrughan.identity)

In [ ]:
#making to check RSA BAsed transactions

def createClientAndTransactionsRSA():
  Azad0 = Client()
  Ravi0 = Client()
  Dinesh0 = Client()
  Ramesh0 = Client()
  Seema0 = Client()
  Vijay0 = Client()
  t1 = Transaction(Azad0 , Ravi0.identity, 15)
  t1.sign_transaction()
  transactionsRSA.append(t1)

  t2 = Transaction(
    Dinesh0,
    Seema0.identity,
    6.0
  )
  t2.sign_transaction()
  transactionsRSA.append(t2)
  t3 = Transaction(
    Ramesh0,
    Vijay0.identity,
    2.0
  )
  t3.sign_transaction()
  transactionsRSA.append(t3)
  t4 = Transaction(
    Seema0,
    Ramesh0.identity,
    4.0
  )
  t4.sign_transaction()
  transactionsRSA.append(t4)
  t5 = Transaction(
    Vijay0,
    Seema0.identity,
    7.0
  )
  t5.sign_transaction()
  transactionsRSA.append(t5)
  t6 = Transaction(
    Ramesh0,
    Seema0.identity,
    3.0
  )
  t6.sign_transaction()
  transactionsRSA.append(t6)
  t7 = Transaction(
    Seema0,
    Dinesh0.identity,
    8.0
  )
  t7.sign_transaction()
  transactionsRSA.append(t7)
  t8 = Transaction(
    Seema0,
    Ramesh0.identity,
    1.0
  )
  t8.sign_transaction()
  transactionsRSA.append(t8)
  t9 = Transaction(
    Vijay0,
    Dinesh0.identity,
    5.0
  )
  t9.sign_transaction()
  transactionsRSA.append(t9)
  t10 = Transaction(
    Vijay0,
    Ramesh0.identity,
    3.0
  )
  t10.sign_transaction()
  transactionsRSA.append(t10)



In [ ]:
def createClientAndTransactionsEDCSA():
  Azad = ClientECDSA()
  Ravi = ClientECDSA()
  Dinesh = ClientECDSA()
  Ramesh = ClientECDSA()
  Seema = ClientECDSA()
  Vijay = ClientECDSA()
  t1 = Transaction(Azad , Ravi.identity, 15)
  t1.sign_transactionECDSA()
  transactionsEDCSA.append(t1)

  t2 = Transaction(
    Dinesh,
    Seema.identity,
    6.0
  )
  t2.sign_transactionECDSA()
  transactionsEDCSA.append(t2)
  t3 = Transaction(
    Ramesh,
    Vijay.identity,
    2.0
  )
  t3.sign_transactionECDSA()
  transactionsEDCSA.append(t3)
  t4 = Transaction(
    Seema,
    Ramesh.identity,
    4.0
  )
  t4.sign_transactionECDSA()
  transactionsEDCSA.append(t4)
  t5 = Transaction(
    Vijay,
    Seema.identity,
    7.0
  )
  t5.sign_transactionECDSA()
  transactionsEDCSA.append(t5)
  t6 = Transaction(
    Ramesh,
    Seema.identity,
    3.0
  )
  t6.sign_transactionECDSA()
  transactionsEDCSA.append(t6)
  t7 = Transaction(
    Seema,
    Dinesh.identity,
    8.0
  )
  t7.sign_transactionECDSA()
  transactionsEDCSA.append(t7)
  t8 = Transaction(
    Seema,
    Ramesh.identity,
    1.0
  )
  t8.sign_transactionECDSA()
  transactionsEDCSA.append(t8)
  t9 = Transaction(
    Vijay,
    Dinesh.identity,
    5.0
  )
  t9.sign_transactionECDSA()
  transactionsEDCSA.append(t9)
  t10 = Transaction(
    Vijay,
    Ramesh.identity,
    3.0
  )
  t10.sign_transactionECDSA()
  transactionsEDCSA.append(t10)

In [ ]:
def createClientAndTransactionsNTRU():
  Azad1 = ClientNTRU()
  Ravi1 = ClientNTRU()
  Dinesh1 = ClientNTRU()
  Ramesh1 = ClientNTRU()
  Seema1 = ClientNTRU()
  Vijay1 = ClientNTRU()
  t1 = Transaction(Azad1 , Ravi1.identity, 15)
  t1.sign_transactionNTRU()
  transactionsNTRU.append(t1)

  t2 = Transaction(
    Dinesh1,
    Seema1.identity,
    6.0
  )
  t2.sign_transactionNTRU()
  transactionsNTRU.append(t2)
  t3 = Transaction(
    Ramesh1,
    Vijay1.identity,
    2.0
  )
  t3.sign_transactionNTRU()
  transactionsNTRU.append(t3)
  t4 = Transaction(
    Seema1,
    Ramesh1.identity,
    4.0
  )
  t4.sign_transactionNTRU()
  transactionsNTRU.append(t4)
  t5 = Transaction(
    Vijay1,
    Seema1.identity,
    7.0
  )
  t5.sign_transactionNTRU()
  transactionsNTRU.append(t5)
  t6 = Transaction(
    Ramesh1,
    Seema1.identity,
    3.0
  )
  t6.sign_transactionNTRU()
  transactionsNTRU.append(t6)
  t7 = Transaction(
    Seema1,
    Dinesh1.identity,
    8.0
  )
  t7.sign_transactionNTRU()
  transactionsNTRU.append(t7)
  t8 = Transaction(
    Seema1,
    Ramesh1.identity,
    1.0
  )
  t8.sign_transactionNTRU()
  transactionsNTRU.append(t8)
  t9 = Transaction(
    Vijay1,
    Dinesh1.identity,
    5.0
  )
  t9.sign_transactionNTRU()
  transactionsNTRU.append(t9)
  t10 = Transaction(
    Vijay1,
    Ramesh1.identity,
    3.0
  )
  t10.sign_transactionNTRU()
  transactionsNTRU.append(t10)

In [ ]:
 print("RSA")
 %timeit createClientAndTransactionsRSA()
 print("EDCSA")
 %timeit createClientAndTransactionsEDCSA()
 print("NTRU")
 %timeit createClientAndTransactionsNTRU()


RSA
1 loop, best of 5: 1.51 s per loop
EDCSA
10 loops, best of 5: 79.8 ms per loop
NTRU
Original message is: 	  [[0, 1, 0, 0, 0, 1, 1], [1, 1, 1, 0, 0, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1], [1, 0, 0, 1, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 0, 0, 1, 1], [0, 1, 0, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 0, 0, 1, 1], [1, 1, 0, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [1, 0, 1, 0, 0, 1, 1], [0, 0, 0, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1], [1, 0, 0, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 1, 1], [0, 0, 1, 1, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [0, 1, 0, 0, 0, 1, 1], [0, 1, 1, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1], [0, 0, 1, 0, 0, 1, 1], [1, 0, 1, 0, 0, 1, 1], [1, 1, 0, 0, 1, 0, 1], 

###In order to add the verifiability of the blocks then adding into the blockchain

In [ ]:
class Block:
    def __init__(self, index, transactions, timestamp, previous_hash, nonce=0):
        self.index = index
        self.verified_transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce

    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

In [ ]:
newblock = Block(1, transactionsNTRU, 23, None)
newblock.compute_hash

<bound method Block.compute_hash of <__main__.Block object at 0x7f510776a450>>

In [ ]:
last_block_hash = ""
Dinesh = ClientECDSA()

t0 = Transaction (
   "Genesis",
   Dinesh.identity,
   500.0
)


block0 = Block(0, [], 23, None)

block0.previous_block_hash = None
Nonce = None

block0.verified_transactions.append (t0)

In [ ]:
digest = hash (block0)
last_block_hash = digest

NITWCoins = []

def dump_blockchain (self):
   print ("Number of blocks in the chain: " + str(len (self)))
   for x in range (len(NITWCoins)):
      block_temp = NITWCoins[x]
      print ("block # " + str(x))
      for transaction in block_temp.verified_transactions:
         display_transaction (transaction)
         print ('--------------')
      print ('=====================================')

In [ ]:
NITWCoins.append (block0)

dump_blockchain(NITWCoins)

Number of blocks in the chain: 1
block # 0
--------------


In [ ]:
def sha256(message):
  return hashlib.sha256(message.encode('ascii')).hexdigest()
def mine(message, difficulty=1):
   assert difficulty >= 1
   prefix = '1' * difficulty
   i = 0
   while(True):
      digest = sha256(str(hash(message)) + str(i))
      if digest.startswith(prefix):
        msg = "after " + str(i) + " iterations found nonce: "+ digest
        print(msg)
        return digest
      else:
        i += 1


In [ ]:
mine("test message ",1)

after 1 iterations found nonce: 1ea9409a32ef11c3b6e3410e905e76a0e6f67a7a81a76470e131c9181648b302


'1ea9409a32ef11c3b6e3410e905e76a0e6f67a7a81a76470e131c9181648b302'

In [ ]:
mine("test message", 4)
mine("test message", 10)
mine("test message", 2)

after 1185 iterations found nonce: 1111d78698c75e514d94e8cd86f4821911704d360d82f6481aff1fb31184f606


In [ ]:
class Blockchain:
    difficulty = 2

    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []

    def create_genesis_block(self):
        genesis_block = Block(0, [], 0, "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    @property
    def last_block(self):
        return self.chain[-1]

    def add_block(self, block, proof):

        previous_hash = self.last_block.hash

        if previous_hash != block.previous_hash:
            return False

        if not Blockchain.is_valid_proof(block, proof):
            return False

        block.hash = proof
        self.chain.append(block)
        return True

    @staticmethod
    def proof_of_work(block):

        block.nonce = 0

        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * Blockchain.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()

        return computed_hash

    def add_new_transaction(self, transaction):
        self.unconfirmed_transactions.append(transaction)

    @classmethod
    def is_valid_proof(cls, block, block_hash):

        return (block_hash.startswith('0' * Blockchain.difficulty) and
                block_hash == block.compute_hash())

    @classmethod
    def check_chain_validity(cls, chain):
        result = True
        previous_hash = "0"

        for block in chain:
            block_hash = block.hash
            
            delattr(block, "hash")

            if not cls.is_valid_proof(block, block_hash) or \
                    previous_hash != block.previous_hash:
                result = False
                break

            block.hash, previous_hash = block_hash, block_hash

        return result

    def mine(self):

        if not self.unconfirmed_transactions:
            return False

        last_block = self.last_block

        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)

        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)

        self.unconfirmed_transactions = []

        return True


In [ ]:
def BlockchainRSA():
  transactions = transactionsRSA
  last_block_hash = None
  last_transaction_index = 0
  block = Block(last_transaction_index, [], 67, None)
  for i in range(3):
    temp_transaction = transactions[i]
    # validate transaction
    # if valid
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1

  block.previous_block_hash = last_block_hash
  block.Nonce = mine (block, 2)
  digest = hash (block)
  NITWCoins.append (block)
  last_block_hash = digest

  # Miner 2 adds a block
  block = Block(last_transaction_index, [], 69, None)

  for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    # validate transaction
    # if valid
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1
  block.previous_block_hash = last_block_hash
  block.Nonce = mine (block, 2)
  digest = hash (block)
  NITWCoins.append (block)
  last_block_hash = digest
  # Miner 3 adds a block


  block = Block(last_transaction_index, [], 69, None)

  for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    #display_transaction (temp_transaction)
    # validate transaction
    # if valid
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1

  block.previous_block_hash = last_block_hash
  block.Nonce = mine (block, 2)
  digest = hash (block)

  NITWCoins.append (block)
  last_block_hash = digest

In [ ]:
def BlockchainEDCSA():
  transactions = transactionsEDCSA
  last_block_hash = None
  last_transaction_index = 0
  block = Block(last_transaction_index, [], 67, None)
  for i in range(3):
    temp_transaction = transactions[i]
    # validate transaction
    # if valid
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1

  block.previous_block_hash = last_block_hash
  block.Nonce = mine (block, 2)
  digest = hash (block)
  NITWCoins.append (block)
  last_block_hash = digest

  # Miner 2 adds a block
  block = Block(last_transaction_index, [], 69, None)

  for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    # validate transaction
    # if valid
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1
  block.previous_block_hash = last_block_hash
  block.Nonce = mine (block, 2)
  digest = hash (block)
  NITWCoins.append (block)
  last_block_hash = digest
  # Miner 3 adds a block


  block = Block(last_transaction_index, [], 69, None)

  for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    #display_transaction (temp_transaction)
    # validate transaction
    # if valid
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1

  block.previous_block_hash = last_block_hash
  block.Nonce = mine (block, 2)
  digest = hash (block)

  NITWCoins.append (block)
  last_block_hash = digest

In [ ]:
def BlockchainNTRU():
  transactions = transactionsNTRU
  last_block_hash = None
  last_transaction_index = 0
  block = Block(last_transaction_index, [], 67, None)
  for i in range(3):
    temp_transaction = transactions[i]
    # validate transaction
    # if valid
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1

  block.previous_block_hash = last_block_hash
  block.Nonce = mine (block, 2)
  digest = hash (block)
  NITWCoins.append (block)
  last_block_hash = digest

  # Miner 2 adds a block
  block = Block(last_transaction_index, [], 69, None)

  for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    # validate transaction
    # if valid
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1
  block.previous_block_hash = last_block_hash
  block.Nonce = mine (block, 2)
  digest = hash (block)
  NITWCoins.append (block)
  last_block_hash = digest
  # Miner 3 adds a block


  block = Block(last_transaction_index, [], 69, None)

  for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    #display_transaction (temp_transaction)
    # validate transaction
    # if valid
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1

  block.previous_block_hash = last_block_hash
  block.Nonce = mine (block, 2)
  digest = hash (block)

  NITWCoins.append (block)
  last_block_hash = digest

In [ ]:
print("RSA")
%timeit BlockchainRSA()

print("EDCSA")
%timeit BlockchainEDCSA()

print("NTRU")
%timeit BlockchainNTRU()


In [ ]:
dump_blockchain(NITWCoins)